In [ ]:
import pandas as pd
import os
import glob

# joining the path and creating list of paths to read all csv files in data2 folder
dataframes = []

for files in glob.glob(r'C:\Users\nebar\Desktop\project\data2\upc12\*.xlsx'):

    print(files)

    data = pd.read_excel(files)

    dataframes.append(data)

data12_df = pd.concat(dataframes, ignore_index=True)

data12_df

In [ ]:
#import the customer detail cleaned data for upc 12
final2_1 = pd.read_excel(r'C:\Users\nebar\Desktop\project\data2\final2_1.xlsx')

final2_1

In [ ]:
#to avoid confusion while creating total connection, rename Newegg Business, amazon and walmart marekt places as Newegg.com, amazon and walmart
data12_df['Merchant'] = data12_df['Merchant'].str.replace('Amazon Marketplace New|Amazon Marketplace Used', 'Amazon.com', regex=True)

data12_df['Merchant']  = data12_df['Merchant'].str.replace('Walmart Marketplace|Wal-Mart.com', 'Walmart', regex = True)
                     
data12_df['Merchant']  = data12_df['Merchant'].str.replace('Newegg Business', 'Newegg.com', regex = True)
                   
data12_df['Merchant'] = data12_df['Merchant'].str.replace('1-800Lighting', '1800lighting.com', regex = True)              

data12_df['Merchant']  = data12_df['Merchant'].str.replace('Younkers(Bon-Ton)', 'Bon-Ton',regex = True)
                       
data12_df

In [ ]:
#check for duplicates using multiple variables
data12_df = pd.DataFrame(data12_df, columns=["EAN", "Title", "Merchant","last_updated"])

duplicate = data12_df[data12_df.duplicated()]

print("Duplicate Rows :")

print(duplicate)

In [ ]:
#drop the duplicates 
data12_df.drop_duplicates()

In [ ]:
#take out the records with null value for the merchant variable
missing_values = pd.notnull(data12_df["Merchant"])

data12_filtered  = data12_df[missing_values]

data12_filtered 

In [ ]:
# Taking out the time part
data12_filtered['last_updated'] = data12_filtered['last_updated'].apply(lambda x: ', '.join(set([part[:10] for part in x.split(', ')])))

data12_filtered

In [ ]:
#to put similar ean and title together to avoid redundancy while creating the total connections
data12_output = data12_filtered.groupby(['EAN','Title']).agg(pd.Series.tolist)

data12_scraped = data12_output.reset_index()

data12_scraped = pd.DataFrame(data12_scraped)

data12_scraped

In [ ]:
#count the total connection for each unique id
data12_scraped ['Total_connections'] = data12_scraped['Merchant'].apply(lambda x: len(set(x)))

data12_scraped

In [ ]:
#to make ean as string
data12_scraped['EAN'] = data12_scraped['EAN'].astype(str)

data12_scraped.dtypes

In [ ]:
#check the customer details datatypes
final2_1.dtypes

In [ ]:
#merge the final cleaned data and datascraped
data12_scraped_output = pd.merge(data12_scraped, final2_1 , left_on = "EAN",right_on = "CONSUMERPACKAGECODE")

data12_scraped_output

In [ ]:
#Arrange the total_connections and total_retailer_connections column 
data12_scraped_new = data12_scraped_output.iloc[0:6775,[0,1,2,3,5,6,7,8,9,10,11,12,13,4]]

data12_scraped_new['Missed_connections'] =  data12_scraped_new['Total_connections']- data12_scraped_new['Total_Retailer_Connections'] 

data12_scraped_new

In [ ]:
#keep the records with missed connections greater than 0
data12_scraped_new = data12_scraped_new[data12_scraped_new.Missed_connections > 0]
 
data12_scraped_new

In [167]:
#export the final file to excel
df = pd.DataFrame(data12_scraped_new)

file_name = 'cleaned12_missed.xlsx'

df.to_excel(file_name,index = False)

print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
